In [27]:
from pathlib import Path
import collections
import re
import plotly.graph_objects as go
import json
print("???")

???


In [35]:
print(f'current path: {Path().absolute()}')       

functions=[
    "base",
    "full",
    "latinize",
    "phonetic-latinize",
    "add-diacritic",
    #"delete-letters",
    "permute-letters",
    #"introduce-extra-letters",
    "confuse-letters",
    "sample-substitute",
    "remove-punctuation",
    #"add-comma",
    #"add-punctuation"
]

def transpose(l):
    return list(map(list, zip(*l)))

def get_model_names(lang):
    single = [f'{lang}_{f}' for f in functions]
    excl = [f'{lang}_excl-{f}' for f in functions]
    # excl.remove('lv_excl-base')
    # excl.remove('lv_excl-delete-letters')
    # excl.remove('lv_excl-introduce-extra-letters')
    # excl.remove('lv_excl-add-comma')
    # excl.remove('lv_excl-full')
    return single
    #return single + excl

def build_table(validation_file, lang, metric):
    table = dict((n, {}) for n in get_model_names(lang))
    exp_path = Path() / 'experiments'

    for f_exp in get_model_names(lang):
        exp = exp_path / f_exp
        for f_eval in functions:
            if f_eval == "full":
                continue
            report_file = exp / 'test' / f'{validation_file}.{f_eval}.{metric}'
            if not report_file.exists():
                table[f_exp][f_eval] = '-'
            else:
                text = report_file.read_text()
                match = re.search(r'([0-9]*\.)?[0-9]+', text)
                table[f_exp][f_eval] = match[0]
        if metric == 'ter':
            file = exp / 'test' / 'overall.ter'
            if file.exists():
                ter = re.search(r'([0-9]*\.)?[0-9]+', file.read_text())[0]
                table[f_exp]['average'] = ter
            else:
                table[f_exp]['average'] = '-'
        if metric == 'bleu':
            if any(x == '-' for x in table[f_exp].values()):
                table[f_exp]['average'] = '-'
                continue
            average = sum(map(lambda x: float(x), table[f_exp].values())) / len(table[f_exp].values())
            table[f_exp]['average'] = round(average, 2)

    return table

lv_bleu_table = build_table('newsdev2017.tc', 'lv', 'bleu')
lv_ter_table = build_table('newsdev2017.tc', 'lv', 'ter')

lt_bleu_table = build_table('dev.tok.tc', 'lt', 'bleu')
lt_ter_table = build_table('dev.tok.tc', 'lt', 'ter')

et_bleu_table = build_table('newsdev2018-enet.tc', 'et', 'bleu')
et_ter_table = build_table('newsdev2018-enet.tc', 'et', 'ter')

lv_big_bleu_table = build_table('newsdev2017.tc', 'lv-big', 'bleu')
lv_big_ter_table = build_table('newsdev2017.tc', 'lv-big', 'ter')

ter_table = lt_bleu_table
bleu_table = lv_big_bleu_table

with open('out2.txt', 'w') as f:
    f.write(json.dumps(bleu_table))
print(json.dumps(bleu_table))

current path: /home/TILDE.LV/arturs.stafanovics/robust_NMT
{"lv-big_base": {"base": "-", "latinize": "-", "phonetic-latinize": "-", "add-diacritic": "-", "permute-letters": "-", "confuse-letters": "-", "sample-substitute": "-", "remove-punctuation": "-", "average": "-"}, "lv-big_full": {"base": "26.71", "latinize": "25.72", "phonetic-latinize": "25.72", "add-diacritic": "26.29", "permute-letters": "26.24", "confuse-letters": "26.15", "sample-substitute": "26.19", "remove-punctuation": "25.15", "average": 26.02}, "lv-big_latinize": {"base": "-", "latinize": "-", "phonetic-latinize": "-", "add-diacritic": "-", "permute-letters": "-", "confuse-letters": "-", "sample-substitute": "-", "remove-punctuation": "-", "average": "-"}, "lv-big_phonetic-latinize": {"base": "-", "latinize": "-", "phonetic-latinize": "-", "add-diacritic": "-", "permute-letters": "-", "confuse-letters": "-", "sample-substitute": "-", "remove-punctuation": "-", "average": "-"}, "lv-big_add-diacritic": {"base": "-", "la

In [4]:
values = [list(ter_table[n].values()) for n in get_model_names('lv')]
values = transpose(values)
print(len(values))

fig = go.Figure(data = go.Table(
    header=dict(values=['model\data'] + functions + ['average']),
    cells=dict(values=[get_model_names('lv')] + values)))

fig.update_layout(
    autosize=False,
    width=2050,
    height=1000,
)
fig.show()

9


In [17]:
values = [list(bleu_table[n].values()) for n in get_model_names('lv')]
values = transpose(values)
print(len(values))

fig = go.Figure(data = go.Table(
    header=dict(values=['model\data'] + functions + ['average']),
    cells=dict(values=[get_model_names('lv')] + values)))

fig.update_layout(
    autosize=False,
    width=2050,
    height=1000,
)
fig.show()


9
